In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_excel(r'/Youtube_Data.xlsx')
df.head()

,brand_found,url,Status
0,Binomo,https://binomo.com/in?a=5022f6c6c258&ac=kashmi...,Relevant
1,Online Mutual Fund Investment in India | Inves...,https://groww.app.link/SuCoMg8SH7,Relevant
2,Rummy Online | Play Indian Rummy Games & Win R...,https://www.rummycircle.com/play-rummy-win-cas...,Relevant
3,Angel BrokingAB-logo-DIY,https://itrade.angelbroking.com/,Relevant
4,VITA - Video Life on the AppStore,https://vita.onelink.me/8u5N/cfcd2050,Relevant


In [15]:
df_title = pd.DataFrame(df, columns=["brand_found"])
df_link = pd.DataFrame(df, columns=["url"])
df_status = pd.DataFrame(df, columns=["status"] )
print("Title Counts :- ", df_title.shape)
print(" Links Counts :- ", df_link.shape)
print("Status Counts :- ", df_status.shape)

Title Counts :-  (1241, 1)
 Links Counts :-  (1241, 1)
Status Counts :-  (1241, 1)


In [18]:
df_titles = df["brand_found"]
df_links = df["url"]
df_statuss = df["Status"] 
print("Title Counts :- ", df_title.shape)
print(" Links Counts :- ", df_link.shape)
print("Status Counts :- ", df_status.shape)

Title Counts :-  (1241, 1)
 Links Counts :-  (1241, 1)
Status Counts :-  (1241, 1)


In [19]:
len(df.index)

1241

In [22]:
df_titles = df["brand_found"].apply(str)
df_titles

0                                                  Binomo
1       Online Mutual Fund Investment in India | Inves...
2       Rummy Online | Play Indian Rummy Games & Win R...
3                                Angel BrokingAB-logo-DIY
4                       VITA - Video Life on the AppStore
                              ...                        
1236                FindMergePointOfLinkedList.cpp GitHub
1237     Dropbox - Sai Ringtones.zip - Simplify your life
1238                            SleekShop.com - Not Found
1239                         geetazaildar.com coming soon
1240     whatis-theplan.org - Registered at Namecheap.com
Name: brand_found, Length: 1241, dtype: object

In [20]:
# let's do Word analyse
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
corpus = []
for i in range(0, 1241):
  review = re.sub('[^a-zA-Z]', ' ', df_titles[i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [25]:
dftitle = pd.DataFrame({'title':corpus})


In [26]:
new_df = pd.concat([df_links,dftitle, df_statuss ], axis=1)
new_df.head()


,url,title,Status
0,https://binomo.com/in?a=5022f6c6c258&ac=kashmi...,binomo,Relevant
1,https://groww.app.link/SuCoMg8SH7,onlin mutual fund invest india invest direct m...,Relevant
2,https://www.rummycircle.com/play-rummy-win-cas...,rummi onlin play indian rummi game win real ca...,Relevant
3,https://itrade.angelbroking.com/,angel brokingab logo diy,Relevant
4,https://vita.onelink.me/8u5N/cfcd2050,vita video life appstor,Relevant


In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()


In [59]:
X = cv.fit_transform(corpus).toarray()
Y = new_df.iloc[:, 2].values

In [60]:
print(X.shape)
print(Y.shape)

(1241, 3334)
(1241,)


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

In [62]:
print(X_train.shape) 
print(X_test.shape) 
print(Y_train.shape) 
print(Y_test.shape)

(992, 3334)
(249, 3334)
(992,)
(249,)


In [63]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000, criterion='entropy')
classifier.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [64]:
y_pred = classifier.predict(X_test)
classifier.score(X_test, Y_test)

0.7028112449799196

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred, normalize=False)

175

In [66]:
accuracy_score(Y_test, y_pred)

0.7028112449799196

In [67]:
# Confusion Matrics
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred)

array([[142,  26],
       [ 48,  33]])

Let's now define the most basic terms, which are whole numbers (not rates):

- true positives (TP): These are cases in which we predicted yes (they have the disease), and they do have the disease.
- true negatives (TN): We predicted no, and they don't have the disease.
- false positives (FP): We predicted yes, but they don't actually have the disease. (Also known as a "Type I error.")
- false negatives (FN): We predicted no, but they actually do have the disease. (Also known as a "Type II error.")

In [68]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

  Irrelevant       0.75      0.85      0.79       168
    Relevant       0.56      0.41      0.47        81

    accuracy                           0.70       249
   macro avg       0.65      0.63      0.63       249
weighted avg       0.69      0.70      0.69       249



- Prevalence: How often does the yes condition actually occur in our sample?
actual yes/total = 105/165 = 0.64
- Recall – What percent of the positive cases did you catch? Recall is the ability of a classifier to find all positive instances.
- F Score: This is a weighted average of the true positive rate (recall) and precision.
- Support is the number of actual occurrences of the class in the specified dataset. Imbalanced support in the training data may indicate structural weaknesses in the reported scores of the classifier and could indicate the need for stratified sampling or rebalancing.

In [69]:
from sklearn.svm import SVC
classifier1 = SVC(kernel = 'linear', random_state = 42)
classifier1.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [70]:
y_pred1 = classifier1.predict(X_test)
classifier1.score(X_test, Y_test)

0.7550200803212851

In [71]:
from xgboost import XGBClassifier
classifier3 = XGBClassifier()
classifier3.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [72]:
y_pred3 = classifier3.predict(X_test)
classifier3.score(X_test, Y_test)

0.714859437751004